In [ ]:
!pip install torchsummary

In [2]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
%matplotlib inline
from torchsummary import summary

import scipy as sp
from functools import partial
from sklearn import metrics
from collections import Counter
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, f1_score
import pretrainedmodels
import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import time 
from tqdm import tqdm_notebook as tqdm
from PIL import Image
train_on_gpu = True
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

import cv2
import albumentations
from albumentations import torch as AT

device = torch.device("cuda:0")

In [3]:
print(os.listdir('../'))

['aptos2019-blindness-detection.zip', 'test_images.zip', 'resnet50-regression.bin', 'train_images.zip', 'aptos']


In [4]:
PATH = "../aptos/"
input_size = (3, 224, 224)

In [5]:
train = pd.read_csv(PATH+'train.csv')
test = pd.read_csv(PATH+'test.csv')
sample_submission = pd.read_csv(PATH+'sample_submission.csv')

In [6]:
class GlassDataset(Dataset):
    def __init__(self, df, datatype='train',transform=transforms.Compose([transforms.CenterCrop(32),transforms.ToTensor()]),y = None):
        self.df = df
        self.datatype = datatype
        self.image_files_list = [f'../aptos/{self.datatype}_images/{i}.png' for i in df['id_code'].values]
        if self.datatype == 'train':
            self.labels = y
        else:
            self.labels = np.zeros((df.shape[0]))
        self.transform = transform

    def __len__(self):
        return len(self.image_files_list)

    def __getitem__(self, idx):
        img_name = self.image_files_list[idx]
        img = cv2.imread(img_name)
        if img.shape[2] == 1:
            img = np.stack((img[..., 0],) * 3, axis=-1)
            
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        image = self.transform(image=img)
        image = image['image']

        img_name_short = self.image_files_list[idx].split('.')[0]

        label = self.labels[idx]
        if self.datatype == 'test':
            return image, label, img_name
        else:
            return image, label


In [7]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = metrics.cohen_kappa_score(y, X_p, weights='quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [1.5, 2.5, 3.5, 4.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        print(self.coef_)
        return self.coef_['x']

In [8]:
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [9]:
class model_(nn.Module):
    def __init__(self, feature=2048, num_classes=1, inchannels=3, model_name='resnet50',pooling='concat'):
        super().__init__()
        self.model_name = model_name
        self.feature = feature
        self.pooling = pooling
        
        if model_name in pretrainedmodels.model_names:
            self.base = pretrainedmodels.__dict__[self.model_name](num_classes=1000, pretrained='imagenet')
            self.base = torch.nn.Sequential(*(list(self.base.children())[:-2]))
            self.base.load_state_dict(torch.load(PATH+"checkpoint/resnet50-regression-0.7.bin"))
        else:
            assert False, "{} is error".format(model_name)
        
        if self.pooling == 'concat':
            print('Concatenated pooling')
            self.ap = nn.AdaptiveAvgPool2d((1,1))
            self.mp = nn.AdaptiveMaxPool2d((1,1))
            self.bn0 = nn.BatchNorm1d(self.feature*2,eps=1e-05, momentum=0.1, affine=True)
            self.dropout0 = nn.Dropout(0.35)
            self.fc1 = nn.Linear(self.feature*2, int(self.feature/2))
            self.bn1 = nn.BatchNorm1d(int(self.feature/2),eps=1e-05, momentum=0.1, affine=True)
            self.dropout1 = nn.Dropout(0.35)
            self.fc2 = nn.Linear(int(self.feature/2), num_classes)
        else:
            if self.pooling == 'average': 
                self.ap = nn.AdaptiveAvgPool2d((1,1))
                print('Average pooling')
            if self.pooling == 'max': 
                self.mp = nn.AdaptiveMaxPool2d((1,1))
                print('Max pooling')
            self.bn0 = nn.BatchNorm1d(self.feature,eps=1e-05, momentum=0.1, affine=True)
            self.dropout0 = nn.Dropout(0.35)
            self.fc1 = nn.Linear(self.feature, int(self.feature/2))
            self.bn1 = nn.BatchNorm1d(int(self.feature/2),eps=1e-05, momentum=0.1, affine=True)
            self.dropout1 = nn.Dropout(0.35)
            self.fc2 = nn.Linear(int(self.feature/2), num_classes)
            
    def forward(self, x):
        x = self.base(x)
        
        if self.pooling == 'concat':
            ap = self.ap(x)
            mp = self.mp(x)
            x = torch.cat((ap,mp),dim=1)
            x = x.view(x.size(0), -1)  #Flatten
            
        if self.pooling == 'max':
            x = self.mp(x)
            x = x.view(x.size(0), -1)
            
        if self.pooling == 'average':
            x = self.ap(x)
            x = x.view(x.size(0), -1)
    
        x = self.bn0(x)
        x = self.dropout0(x)
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = self.dropout1(x)         
        x = self.fc2(x)
        
        return x


In [10]:
model = model_(pooling='concat')
model = model.to(device)
summary(model, input_size=input_size)

Concatenated pooling
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-

In [11]:
data_transforms = albumentations.Compose([
    albumentations.Resize(input_size[1], input_size[2]),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Transpose(p=0.5),
    albumentations.Flip(p=0.0),
    albumentations.OneOf([
        albumentations.CLAHE(clip_limit=2), albumentations.IAASharpen(), albumentations.IAAEmboss(), 
        albumentations.RandomBrightness(), albumentations.RandomContrast(),
        albumentations.JpegCompression(), albumentations.Blur(), albumentations.GaussNoise()], p=0.5), 
    albumentations.HueSaturationValue(p=0.5), 
    albumentations.ShiftScaleRotate(shift_limit=0.15, scale_limit=0.15, rotate_limit=45, p=0.5),
    albumentations.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
    AT.ToTensor()
    ])

In [12]:
batch_size = 32
num_workers = 4
lr = 0.01
y = train['diagnosis']

In [13]:
dataset = GlassDataset(df=train, datatype='train',transform=data_transforms,y=y)

tr, val = train_test_split(train.diagnosis, stratify=train.diagnosis, test_size=0.2)
train_sampler = SubsetRandomSampler(list(tr.index))
valid_sampler = SubsetRandomSampler(list(val.index))

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(dataset, batch_size=1, sampler=valid_sampler, num_workers=num_workers)

#optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.99), weight_decay=0.0002)

optimizer = optim.Adam([{'params': model.fc1.parameters(),'lr':0.0064},
                        {'params': model.fc2.parameters(),'lr':0.0064},
                        {'params': model.base.parameters(), 'lr': 1e-4}], 
                       weight_decay=0.0002)

scheduler = lr_scheduler.StepLR(optimizer, step_size=10)

In [ ]:
best_kappa = 0.7
patience = 5
# current number of epochs, where validation loss didn't increase
p = 0
# whether training should be stopped
stop = False

since = time.time()
criterion = nn.MSELoss()
num_epochs = 100
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    scheduler.step()
    model.train()
    running_loss = 0.0
    tk0 = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    for bi, (data, target) in enumerate(tk0):
        inputs = data
        labels = target.view(-1, 1)
        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        counter += 1
        tk0.set_postfix(loss=(running_loss / (counter * train_loader.batch_size)))
    epoch_loss = running_loss / len(train_loader)
    print('Training Loss: {:.4f}'.format(epoch_loss))
    
    model.eval()
    val_loss = []
    val_auc = []
    
    val_target = np.zeros(len(val))
    val_pred = np.zeros(len(val))
    tk1 = tqdm(valid_loader)
    for batch_i, (data, target) in enumerate(tk1):
        inputs = data
        labels = target.view(-1, 1)
        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        val_loss.append(loss.item()) 
        a = target.data.cpu().numpy()
        b = outputs.detach().cpu().numpy()
        val_target[batch_i] = a
        val_pred[batch_i] = b
        
    optR = OptimizedRounder()
    optR.fit(val_pred, val_target)
    coefficients = optR.coefficients()
    valid_predictions = optR.predict(val_pred, coefficients)
    
    accuracy = accuracy_score(val_target, valid_predictions)
    print('Accuracy : {:.4f}'.format(accuracy))
    kappa = metrics.cohen_kappa_score(val_target, valid_predictions, labels=None, weights=None, sample_weight=None)
    print('Kappa : {:.4f}'.format(kappa))
    f1 = f1_score(val_target, valid_predictions, labels=None, pos_label=1, average='weighted', sample_weight=None)
    print('F1 score: {:.4f}'.format(f1))
    if best_kappa<kappa:
        best_kappa = kappa
        torch.save(model.state_dict(), "../aptos/checkpoint/resnet50-"+format(kappa)+".bin")
        print('Model save')
    if best_kappa > kappa:
        p += 1
        print(f'{p} epochs oµf increasing val loss')
        if p > patience:
            print('Stopping training')
            stop = True
            break        
            
    if stop:
        break

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
#torch.save(model.state_dict(), "../aptos/checkpoint/resnet50"+format(epoch_loss)+"-regression.bin")

Epoch 0/99
----------



Training Loss: 61.4392



 final_simplex: (array([[0.57697895, 2.80547232, 3.78523498, 5.27120441],
       [0.5769791 , 2.80548058, 3.78515094, 5.27115165],
       [0.57697036, 2.80550243, 3.7851676 , 5.27111111],
       [0.57698328, 2.805481  , 3.78516765, 5.27111419],
       [0.57696903, 2.80548462, 3.78517707, 5.27115308]]), array([-0.44801707, -0.44801707, -0.44801707, -0.44801707, -0.44801707]))
           fun: -0.4480170680215848
       message: 'Optimization terminated successfully.'
          nfev: 122
           nit: 46
        status: 0
       success: True
             x: array([0.57697895, 2.80547232, 3.78523498, 5.27120441])
Accuracy : 0.3834
Kappa : 0.1816
F1 score: 0.4156
1 epochs oµf increasing val loss
Epoch 1/99
----------



Training Loss: 29.4360



 final_simplex: (array([[0.01031505, 0.47938555, 5.2385328 , 8.06663449],
       [0.01032671, 0.47938018, 5.23856713, 8.06658774],
       [0.01032154, 0.47937079, 5.23858253, 8.06657738],
       [0.01029919, 0.47934181, 5.23860701, 8.06665066],
       [0.01031014, 0.47941564, 5.23857543, 8.06657921]]), array([-0.78926709, -0.78926709, -0.78926709, -0.78926709, -0.78926709]))
           fun: -0.7892670872957217
       message: 'Optimization terminated successfully.'
          nfev: 122
           nit: 42
        status: 0
       success: True
             x: array([0.01031505, 0.47938555, 5.2385328 , 8.06663449])
Accuracy : 0.7435
Kappa : 0.5930
F1 score: 0.6872
2 epochs oµf increasing val loss
Epoch 2/99
----------


...


Training Loss: 22.3200



 final_simplex: (array([[0.74299007, 1.19067239, 2.20340718, 8.27367109],
       [0.74298859, 1.19064394, 2.2034073 , 8.27374462],
       [0.74298137, 1.1906823 , 2.20341869, 8.27368516],
       [0.74298689, 1.1906597 , 2.20342108, 8.27369522],
       [0.74298398, 1.19063719, 2.20340665, 8.27373288]]), array([-0.80297053, -0.80297053, -0.80297053, -0.80297053, -0.80297053]))
           fun: -0.8029705283351793
       message: 'Optimization terminated successfully.'
          nfev: 147
           nit: 62
        status: 0
       success: True
             x: array([0.74299007, 1.19067239, 2.20340718, 8.27367109])
Accuracy : 0.6235
Kappa : 0.4521
F1 score: 0.6142
4 epochs oµf increasing val loss
Epoch 4/99
----------



Training Loss: 20.3973



 final_simplex: (array([[1.18715731, 1.32909153, 4.11851403, 4.91462383],
       [1.18710632, 1.32906048, 4.11860838, 4.91458052],
       [1.18712844, 1.32910969, 4.11856116, 4.91459996],
       [1.18712485, 1.32903752, 4.11854594, 4.91464979],
       [1.18713111, 1.32903126, 4.11856723, 4.91464774]]), array([-0.77678361, -0.77678361, -0.77678361, -0.77678361, -0.77678361]))
           fun: -0.7767836132650829
       message: 'Optimization terminated successfully.'
          nfev: 119
           nit: 41
        status: 0
       success: True
             x: array([1.18715731, 1.32909153, 4.11851403, 4.91462383])
Accuracy : 0.7422
Kappa : 0.5839
F1 score: 0.6710
5 epochs oµf increasing val loss
Epoch 5/99
----------



Training Loss: 20.2054


In [ ]:
dataset = GlassDataset(df=train, datatype='train',transform=data_transforms,y=y)

tr, val = train_test_split(train.diagnosis, stratify=train.diagnosis, test_size=0.99)
train_sampler = SubsetRandomSampler(list(tr.index))
valid_sampler = SubsetRandomSampler(list(val.index))

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(dataset, batch_size=1, sampler=valid_sampler, num_workers=num_workers)

torch.cuda.empty_cache()

model.eval()
val_target = np.zeros(len(val))
val_pred = np.zeros(len(val))
tk1 = tqdm(valid_loader)
for batch_i, (data, target) in enumerate(tk1):
    inputs = data
    labels = target.view(-1, 1)
    inputs = inputs.to(device, dtype=torch.float)
    labels = labels.to(device, dtype=torch.float)
    outputs = model(inputs)
    loss = criterion(outputs, labels)

    val_loss.append(loss.item()) 
    a = target.data.cpu().numpy()
    b = outputs.detach().cpu().numpy()
    val_target[batch_i] = a
    val_pred[batch_i] = b
    

In [ ]:
optR = OptimizedRounder()
optR.fit(val_pred, val_target)
coefficients = optR.coefficients()
valid_predictions = optR.predict(val_pred, coefficients)
    
accuracy = accuracy_score(val_target, valid_predictions)
print('Accuracy : {:.4f}'.format(accuracy))
kappa = metrics.cohen_kappa_score(val_target, valid_predictions, labels=None, weights=None, sample_weight=None)
print('Kappa : {:.4f}'.format(kappa))

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((input_size[1], input_size[2])),
    AT.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_set = GlassDataset(df=test, datatype='test', transform=data_transforms_test)
test_data_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False, num_workers=4)
test_preds = np.zeros((len(test_dataset), 1))
tk0 = tqdm(test_data_loader)
for i, x_batch in enumerate(tk0):
    x_batch = x_batch["image"]
    pred = model(x_batch.to(device))
    test_preds[i * 32:(i + 1) * 32] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)

In [ ]:
coef = [0.5, 1.5, 2.5, 3.5]

for i, pred in enumerate(test_preds):
    if pred < coef[0]:
        test_preds[i] = 0
    elif pred >= coef[0] and pred < coef[1]:
        test_preds[i] = 1
    elif pred >= coef[1] and pred < coef[2]:
        test_preds[i] = 2
    elif pred >= coef[2] and pred < coef[3]:
        test_preds[i] = 3
    else:
        test_preds[i] = 4


sample = pd.read_csv(PATH+"sample_submission.csv")
sample.diagnosis = test_preds.astype(int)
sample.to_csv("submission.csv", index=False)

In [ ]:
sample